In [26]:
import yaml
import requests
import json
import numpy as np
import pandas as pd
import seaborn as sns
from requests_oauthlib import OAuth2Session
from pandas.io.json import json_normalize

In [2]:
# get secrets
client_id=""
secret=""
with open("../credentials.yaml", "r") as stream:
    try:
        data = yaml.safe_load(stream)
        client_id=data['client_id']
        secret=data['secret']
        print("Secrets obtained")
    except yaml.YAMLError as exc:
        print(exc)

Secrets obtained


In [6]:
#https://www.grace-dev.com/python-apis/strava-api/
#Get Strava data
redirect_uri='http://localhost'

#Authorizion url
request_url = f'http://www.strava.com/oauth/authorize?client_id={client_id}' \
              f'&response_type=code&redirect_uri={redirect_uri}' \
              f'&approval_prompt=force' \
              f'&scope=profile:read_all,activity:read_all'

# User prompt showing the Authorization URL
# and asks user for the code

print('Click here:', request_url)
print('Please authorize the app and copy&paste below the generated code!')
print('P.S: you can find the code in the URL')
code = input('Insert the code from the url: ')

# Get the access token
token=requests.post(url='https://www.strava.com/api/v3/oauth/token',
                    data = {'client_id': client_id,
                    'client_secret': secret,
                    'code': code,
                    'grant_type': 'authorization_code'}
)

#Save json response as a variable
strava_token = token.json()
print(strava_token)

with open('strava_token.json', 'w') as outfile:
    json.dump(strava_token, outfile)

Click here: http://www.strava.com/oauth/authorize?client_id=48539&response_type=code&redirect_uri=http://localhost&approval_prompt=force&scope=profile:read_all,activity:read_all
Please authorize the app and copy&paste below the generated code!
P.S: you can find the code in the URL
{'token_type': 'Bearer', 'expires_at': 1675230143, 'expires_in': 21494, 'refresh_token': '29d2cc9eb15a2287adf0a8b00ec1fb45842027ad', 'access_token': '43354bc2579c794dd168b0bf75cec772d7b05484', 'athlete': {'id': 49433241, 'username': None, 'resource_state': 2, 'firstname': 'Deanna', 'lastname': 'Church', 'bio': None, 'city': None, 'state': None, 'country': None, 'sex': 'F', 'premium': False, 'summit': False, 'created_at': '2020-01-01T00:11:49Z', 'updated_at': '2023-01-31T23:41:43Z', 'badge_type_id': 0, 'weight': 0.0, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/49433241/13373524/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/49433241/13373524/1/la

In [13]:
#Get athlete data
with open('strava_token.json', 'r') as token:
    data=json.load(token)

#Get the access token
access_token=data['access_token']

#Build the API url to get athlete info
athlete_url=f"https://www.strava.com/api/v3/athlete?" \
            f"access_token={access_token}"

#get the response in JSON format
response=requests.get(athlete_url)
athlete=response.json()

#print some stuff
print('RESTful API: ', athlete_url)
print('=' * 5, 'ATHLETE INFO', '=' * 5)
print('Name:', athlete['firstname'] + athlete['lastname'])
print('Sex: ', athlete['sex'])
print('City:', athlete['city'], athlete['country'])
print('Strava athlete from: ', athlete['created_at'])

{'id': 49433241, 'username': None, 'resource_state': 3, 'firstname': 'Deanna', 'lastname': 'Church', 'bio': None, 'city': None, 'state': None, 'country': None, 'sex': 'F', 'premium': False, 'summit': False, 'created_at': '2020-01-01T00:11:49Z', 'updated_at': '2023-01-31T23:41:43Z', 'badge_type_id': 0, 'weight': 0.0, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/49433241/13373524/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/49433241/13373524/1/large.jpg', 'friend': None, 'follower': None, 'blocked': False, 'can_follow': True, 'follower_count': 12, 'friend_count': 13, 'mutual_friend_count': 0, 'athlete_type': 0, 'date_preference': '%m/%d/%Y', 'measurement_preference': 'feet', 'clubs': [], 'ftp': None, 'bikes': [], 'shoes': [], 'is_winback_via_upload': False, 'is_winback_via_view': False}
RESTful API:  https://www.strava.com/api/v3/athlete?access_token=43354bc2579c794dd168b0bf75cec772d7b05484
===== ATHLETE INFO =====
Name: D

In [14]:
with open('strava_token.json', 'r') as token:
    data=json.load(token)

#Build the API url to get activities
activities_url=f'https://www.strava.com/api/v3/athlete/activities?' \
               f'access_token={access_token}'

print('RESTful API:', activities_url)

RESTful API: https://www.strava.com/api/v3/athlete/activities?access_token=43354bc2579c794dd168b0bf75cec772d7b05484


In [32]:
#get response in JSON format
response=requests.get(activities_url)
activity=response.json()

#print(activity)
#act_norm=json_normalize(activity)
#print(act_norm)
act_df=pd.DataFrame(activity)
act_df.head()



,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,...,average_cadence,average_watts,max_watts,weighted_average_watts,kilojoules,device_watts,average_heartrate,max_heartrate,elev_high,elev_low
0,2,"{'id': 49433241, 'resource_state': 1}",30 min Strength Roll Call: Upper Body with Adr...,0.0,1800,1800,0.0,Workout,Workout,8482308528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"{'id': 49433241, 'resource_state': 1}",10 min Strength Roll Call: Core with Callie Gu...,0.0,600,600,0.0,Workout,Workout,8482163209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"{'id': 49433241, 'resource_state': 1}",20 min Low Impact Ride with Ally Love,9536.4,1200,1200,0.0,Ride,Ride,8482053809,...,83.6,127.6,199.0,133.0,153.1,True,135.0,161.0,NaN,NaN
3,2,"{'id': 49433241, 'resource_state': 1}",Lunch Walk,1550.1,1661,1753,16.6,Walk,Walk,8482318454,...,NaN,NaN,NaN,NaN,NaN,NaN,98.5,112.0,1642.4,1630.2
4,2,"{'id': 49433241, 'resource_state': 1}",10 min Lower Body Stretch with Matty Maggiacomo,0.0,540,540,0.0,Workout,Workout,8476871085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
